In [ ]:
from selenium import webdriver
import time 
import pandas as pd
import numpy as np

In [ ]:
path ="C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(path)
driver.get("https://food.jumia.dz/restaurants")
try:    
    driver.find_element_by_class_name("popup-close").click()
except:
    print("hello")
# driver.find_element_by_class_name("close-popup.mla").click()

In [ ]:
name = "shop_checkout_type[orderMetaData][customerAddress][addressLine1]" 
search = driver.find_element_by_name(name)
search.send_keys("gardern city")
time.sleep(3)
driver.find_element_by_class_name("geo-option").click()
time.sleep(3)
driver.find_element_by_class_name("button.mvs.fw.is-success").click()

In [ ]:
articles = driver.find_elements_by_tag_name("article")
data_basic = []

In [ ]:
for i in range(len(articles)):
    name_vend = articles[i].find_element_by_class_name("name").text
    bottom = articles[i].find_element_by_class_name("bottom-info").text
    link = articles[i].find_element_by_tag_name("a").get_attribute("href")
    data_basic.append([name_vend, bottom, link])
    
df = pd.DataFrame(data_basic, columns = ["name", "bottom", "link"])
df.to_csv("data_basic.csv", index = False)

In [ ]:
df = pd.DataFrame(columns = ["name", "bottom", "link"])
df = pd.read_csv("data_basic.csv")

In [ ]:
data_menu = pd.DataFrame(columns=["restaurant", "resume", "section", "product", "price"])

In [ ]:
for rest in range(len(df)):
    link = df.iloc[rest]["link"][21:]
    nom = df.iloc[rest]["name"]
    bottom = df.iloc[rest]["bottom"]
    path = "//a[@href = '"+link+"']"
    try :
        driver.find_element_by_xpath(path).click()
        print(nom)
    except:
        continue
    time.sleep(.1)
    category_sections = driver.find_elements_by_xpath("//section[@class = 'menu-category-section mtxxl' ]")
    i = 0
    for section in category_sections:
        x = section.find_element_by_tag_name("div").text
        articles = section.find_elements_by_tag_name("article")
        for article in articles:
            product = driver.find_elements_by_xpath("//h3/span")[i].text
            price = driver.find_elements_by_xpath("//a[starts-with(@class, 'product-price-anchor')]/span[@class = 'mlxs fsh-0']")[i].text
            a = {
                "restaurant" : nom,
                "resume" : bottom,
                "section" : x,
                "product" : product,
                "price": price
            }
            data_menu = data_menu.append(a, ignore_index = True)
            i +=1
    driver.back()
    time.sleep(.1)

In [84]:
data_menu.to_csv("data_menu.csv", index = False)

In [ ]:
def section_trt(x):
    x = x.replace("NOS", "")
    if "TACOS" in x: return 'TACOS'
    if "PIZZA" in x: return "PIZZA"
    if "BURGER" in x: return "BURGER"
    if "SANDWICHS" in x: return "SANDWICHS"
    if 'SALADES' in  x: return 'SALADES'
    if "PANINIS" in x : return "PANINIS"
    if "SASHIMI" in x or "ROLL" in x or "MAKI" in x: return "THAI"
    if "BOISSONS" in x or 'MOJITOS' in x: return "BOISSONS"
    if "ENTR" in x : return "ENTREES"
    if "P" in x and "TES" in x : return "PATES"
    if "LES PLUS COMMANDÉS"in x: return "TOP"
    return x


In [ ]:
data_menu["section_trt"] = data_menu["section"].apply(section_trt)

In [ ]:
np.unique(data_menu["section_trt"].values)